In [1]:
import jieba
import numpy as np 
import collections
import random

In [2]:
def get_ch_lable(txt_file):  
    labels= ""
    print(txt_file)
    with open(txt_file, 'rb') as f:
        for label in f: 
            labels =labels+label.decode('utf-8')
    return labels

In [3]:
def get_split_word(training_data):
    seg_list = jieba.cut(training_data)
    training_ci = " ".join(seg_list)
    training_ci = training_ci.split()
    training_ci = np.array(training_ci)
    training_ci = np.reshape(training_ci, [-1, ])
    return training_ci

In [4]:
def build_dataset(words, top_n):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(top_n - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [5]:
data_index = 0

In [43]:
def generate_batch(data,batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)

    if data_index + span > len(data):
        data_index = 0

    buffer.extend(data[data_index : data_index + span])
    print(buffer)
    data_index += span

    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]

            buffer.append(data[data_index])
            data_index += 1

#     data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [ ]:
def run():
    tf.reset_default_graph()

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [50]:
if __name__ == "__main__":
    training_data = get_ch_lable('./test.txt')
    jieba_data = get_split_word(training_data)
    training_label, count, dictionary, words = build_dataset(jieba_data, 350)
    # print(training_label, count, dictionary, words)
    words_size = len(dictionary)
    print('Sample data', training_label[:10], [words[i] for i in training_label[:10]])
    batch, labels = generate_batch(training_label,batch_size=8, num_skips=2, skip_window=1)
    print(batch)
    print('------')
    print(labels)
    print('------')
    
    for i in range(8):# 取第一个字，后一个是标签，再取其前一个字当标签，
        print(batch[i], words[batch[i]], '->', labels[i, 0], words[labels[i, 0]])
        
    batch_size = 128
    embedding_size = 128  # Dimension of the embedding vector.
    skip_window = 1       # How many words to consider left and right.
    num_skips = 2         # How many times to reuse an input to generate a label.

    valid_size = 16     # Random set of words to evaluate similarity on.
    valid_window =np.int32( words_size/2 ) # Only pick dev samples in the head of the distribution.
    print("valid_window",valid_window)
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)#0-words_size/2,中的数取16个。不能重复。
    num_sampled = 64    # Number of negative examples to sample.

./test.txt
Sample data [25, 132, 32, 26, 27, 133, 8, 9, 80, 134] ['人体', '阴阳', '与', '电能', '阴', '应该', '是', '身体', '里', '内在']
deque([4, 162, 17], maxlen=3)
[162  17   3 163   7  85   9   2]
------
[[ 17]
 [162]
 [163]
 [  3]
 [163]
 [  9]
 [ 85]
 [164]]
------
162 坏 -> 17 了
17 了 -> 162 坏
3 。 -> 163 对于
163 对于 -> 3 。
7 运动 -> 163 对于
85 对 -> 9 身体
9 身体 -> 85 对
2 的 -> 164 影响
valid_window 175


4 就 -> 1 ，
16 相当于 -> 50 机器
50 机器 -> 16 相当于
17 了 -> 3 。
3 。 -> 17 了
146 举个 -> 147 最
147 最 -> 84 简单
84 简单 -> 147 最


In [32]:
print(words[33])

成
